In [ ]:
# # ユーザー認証
# from google.colab import auth
# auth.authenticate_user()

#Google setting
from google import colab
colab.drive.mount('/content/gdrive')

#Directory setting
b_dir='./gdrive/My Drive/AI_Seminar/NLP_1/' # Setting working directory
dataset_dir=b_dir+'dataset/'

In [ ]:
# # データのダウンロード
# !mkdir dataset
# !wget "https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S" -O ./dataset/train.csv
# !wget "https://drive.google.com/uc?export=download&id=1Mdyv3DPcb1AncxE7H739QWyPkMqcdjqI" -O ./dataset/dev.csv
# !wget "https://drive.google.com/uc?export=download&id=1yHKfDrs6ZAx4fizToTF7GL7ROvGk7Ry5" -O ./dataset/test.csv

mkdir: cannot create directory ‘dataset’: File exists
--2021-01-10 08:04:13--  https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S
Resolving drive.google.com (drive.google.com)... 172.217.9.206, 2607:f8b0:4004:806::200e
Connecting to drive.google.com (drive.google.com)|172.217.9.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qus2qn12q0f3oim2hpab21b4mflaptnc/1610265825000/12400649359575842542/*/1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S?e=download [following]
--2021-01-10 08:04:14--  https://doc-0s-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qus2qn12q0f3oim2hpab21b4mflaptnc/1610265825000/12400649359575842542/*/1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S?e=download
Resolving doc-0s-08-docs.googleusercontent.com (doc-0s-08-docs.googleusercontent.com)... 172.217.7.129, 2607:f8b0:4004:805::2001
Connecting to do

In [ ]:
import csv
import random

In [ ]:
# データの読み込み
train = list(csv.reader(open(dataset_dir+"train.csv")))[1:] # 訓練データ．1行目はヘッダー行なので除外
val = list(csv.reader(open(dataset_dir+"dev.csv")))[1:] # 検証用データ．1行目はヘッダー行なので除外
test = list(csv.reader(open(dataset_dir+"test.csv")))[1:] # 最終評価用データ．1行目はヘッダー行なので除外

# 訓練データのみシャッフル
random.seed(42)
random.shuffle(train)

train_movienames, train_titles, train_texts, train_ratings = zip(*train) # 訓練データ．
val_movienames, val_titles, val_texts, val_ratings = zip(*val) # 検証用データ
test_movienames, test_titles, test_texts = zip(*test) # 最終評価用データ．rating は付与されていない
y_train = [int(rating) for rating in train_ratings] # rating が文字列で付与されているので，int 型に変換
y_val = [int(rating) for rating in val_ratings] # rating が文字列で付与されているので，int 型に変換

In [ ]:
# ツールのインストール
!pip install tensorflow_text

In [ ]:
'''
-----------------------------------------------------
Universal Sentence Encoder
title と text を用いて rating を予測する分類器を作成
-----------------------------------------------------
'''

import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
# use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
!pip install tqdm

import numpy as np
from tqdm import tqdm_notebook as tqdm

In [ ]:
X_titles_train = use(train_titles)
X_titles_val = use(val_titles)
X_titles_test = use(test_titles)

In [ ]:
# _X_texts_train = np.empty([1, 512])
# _X_texts_val = np.empty([1, 512])
# _X_texts_test = np.empty([1, 512])
# num = int(len(train_texts)/100)
# for i in tqdm(range(num)):
#     tmp_X_train = use(np.array_split(train_texts, num)[i])
#     tmp_X_val = use(np.array_split(val_texts, num)[i])
#     tmp_X_test = use(np.array_split(test_texts, num)[i])
#     _X_texts_train = np.concatenate([_X_texts_train, tmp_X_train])
#     _X_texts_val = np.concatenate([_X_texts_val, tmp_X_val])
#     _X_texts_test = np.concatenate([_X_texts_test, tmp_X_test])

In [ ]:
X_texts_train = []
X_texts_val = []
X_texts_test = []
for text in tqdm(train_texts):
    X_texts_train.append(use(text))
for text in tqdm(val_texts):
    X_texts_val.append(use(text))
for text in tqdm(test_texts):
    X_texts_test.append(use(text))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
# X_movienames_train = []
# X_movienames_val = []
# X_movienames_test = []
# for text in tqdm(train_movienames):
#     X_texts_train.append(use(text))
# for text in tqdm(val_movienames):
#     X_texts_val.append(use(text))
# for text in tqdm(test_movienames):
#     X_texts_test.append(use(text))

In [ ]:
print(np.array(X_titles_train).shape)
print(np.array(X_texts_train).shape)
# print(np.array(X_movienames_train).shape)

(30000, 512)
(30000, 1, 512)


In [ ]:
# X_texts_train = use(train_texts)  # ResourceExhaustedError
# X_texts_val = use(val_texts)  # ResourceExhaustedError
# X_texts_test = use(test_texts)  # ResourceExhaustedError

X_titles_train = np.array(X_titles_train)
X_titles_val = np.array(X_titles_val)
X_titles_test = np.array(X_titles_test)
X_texts_train = np.squeeze(np.array(X_texts_train))
X_texts_val = np.squeeze(np.array(X_texts_val))
X_texts_test = np.squeeze(np.array(X_texts_test))
# X_movienames_train = np.squeeze(np.array(X_movienames_train))
# X_movienames_val = np.squeeze(np.array(X_movienames_val))
# X_movienames_test = np.squeeze(np.array(X_movienames_test))

X_train = np.concatenate((X_titles_train, X_texts_train), axis=1)
X_val = np.concatenate((X_titles_val, X_texts_val), axis=1)
X_test = np.concatenate((X_titles_test, X_texts_test), axis=1)

# X_train = np.concatenate((X_movienames_train, X_train), axis=1)
# X_val = np.concatenate((X_movienames_val, X_val), axis=1)
# X_test = np.concatenate((X_movienames_test, X_test), axis=1)

# print(X_movienames_train.shape)
print(X_titles_train.shape)
print(X_texts_train.shape)
print(X_train)

(30000, 512)
(30000, 512)
[[-0.02009209 -0.02410404 -0.07918149 ... -0.00445453  0.00782959
  -0.02272757]
 [ 0.03217473 -0.0504187   0.02995347 ... -0.05616933 -0.07436803
  -0.01612947]
 [ 0.03381559 -0.04979878 -0.015561   ... -0.01587693 -0.08548185
  -0.01078122]
 ...
 [ 0.04673385 -0.02811936  0.00942355 ... -0.04558867  0.01358751
  -0.08832002]
 [-0.01169779 -0.09005183  0.01869082 ... -0.0632165  -0.03604158
   0.04751801]
 [ 0.00746338  0.05143991 -0.05516728 ... -0.04008516 -0.00827918
  -0.01328531]]


In [ ]:
# '''
# -----------------------------------------------------
# 分類器の訓練，検証
# -----------------------------------------------------
# '''

# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score

# # Grid Search
# params = {'max_depth': [3, 4, 5, 6,7],
#             'min_child_weight': [0.5, 0.8, 1, 2, 3],
#             'n_estimators': [50, 100, 200, 300, 500],
#             'subsample': [0.8, 0.95, 1],
#             'colsample_bytree': [1.0]
#     }

# clf = GridSearchCV(xgb.XGBClassifier(random_state=0, tree_method='gpu_hist'),
#                    params, scoring ='accuracy', cv=5, n_jobs = -1)
# clf = clf.fit(X_train, y_train)

# print("training score : {}".format(clf.best_score_))
# print("validation score : {}".format(clf.score(X_val, y_val)))
# y_test_pred = clf.predict(X_test)

In [ ]:
# '''
# -----------------------------------------------------
# 分類器の訓練，検証
# -----------------------------------------------------
# '''

# import xgboost as xgb
# from sklearn.metrics import accuracy_score

# clf = xgb.XGBClassifier(random_state=0, tree_method='gpu_hist')
# clf.fit(X_train, y_train, verbose=True)
# # eval_metric : https://xgboost.readthedocs.io/en/latest/parameter.html

# print("validation score : {}".format(accuracy_score(y_val, clf.predict(X_val))))
# y_test_pred = clf.predict(X_test)

In [ ]:
# result = "\n".join([str(rating) for rating in y_test_pred])

In [ ]:
# open("./dataset/test_pred_titles-texts_xgboost.txt", "w").write(result)

In [ ]:
'''
-----------------------------------------------------
分類器の訓練
-----------------------------------------------------
'''

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 分類器の訓練
best_c, best_score = 0, 0
for c in [0.1, 1, 10, 100]:
    classifier = LogisticRegression(C=c, max_iter=1000)
    classifier.fit(X_train, y_train)
    val_acc = accuracy_score(y_val, classifier.predict(X_val))
    if best_score < val_acc:
        best_score = val_acc
        best_c = c
    print("Validation accuracy = %1.3f\tC = %s" % (val_acc, str(c)))
print("Best parameter: C = %s" % str(best_c))

Validation accuracy = 0.439	C = 0.1
Validation accuracy = 0.455	C = 1
Validation accuracy = 0.467	C = 10
Best parameter: C = 10


In [ ]:
'''
-----------------------------------------------------
評価
-----------------------------------------------------
'''

classifier = LogisticRegression(C=best_c, max_iter=1000)
classifier.fit(X_train, y_train)
# y_test_pred = classifier.predict(X_test)
y_test_predProba = classifier.predict_proba(X_test)

In [ ]:
# result = "\n".join([str(rating) for rating in y_test_pred])
result = "\n".join([str(rating) for rating in y_test_predProba])

In [ ]:
# open("./dataset/test_pred_titles-texts_lr.txt", "w").write(result)
open(dataset_dir+"test_predProba_titles-texts_lr.txt", "w").write(result)

1999

In [ ]:
# print("Default XGBoost Training Accuracy Score : {}".format(accuracy_score(y_train, clf.predict(X_train))))
print("C=1 LogisticRegression Training Accuracy Score : {}".format(accuracy_score(y_train, classifier.predict(X_train))))

C=1 LogisticRegression Training Accuracy Score : 0.5294
